In [2]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

/Users/stjiao/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
data = pd.read_csv("train_detect_sent2.csv").reset_index(drop=True)

In [4]:
data.shape

(1314, 17)

In [5]:
data.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,euclidean_dis2,minkowski_dis,chebyshev_dis,pred_idx_cos,pred_idx_euc,pred_idx_min,pred_idx_che
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.1095634 0.1142294 0.04428943 ... 0.0...,"[0.43262481689453125, 0.36864835023880005, 0.3...","[14.108408, 15.0115595, 18.043663, 13.160972, ...","[3.7561161518096924, 3.8744754791259766, 4.247...","[175.40135, 182.2885, 207.47821, 169.22188, 16...","[0.32886288, 0.3149855, 0.28589985, 0.26730207...",5,5,5,4
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.10951651 0.11030624 0.05210008 ... -0.0...,"[0.454074501991272, 0.32262009382247925, 0.355...","[12.889506, 12.285219, 16.843704, 8.361172, 11...","[3.590195894241333, 3.5050275325775146, 4.1041...","[167.5769, 162.31854, 193.53157, 130.61359, 16...","[0.31238672, 0.27531803, 0.2836463, 0.3119438,...",3,3,3,4
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.00262268 0.14930709 0.03063983 ... 0.0...,"[0.39596283435821533, 0.315723180770874, 0.346...","[11.590837, 12.180486, 16.56459, 7.4446793, 9....","[3.404531717300415, 3.4900553226470947, 4.0699...","[159.62509, 163.19696, 196.51852, 121.326935, ...","[0.26670542, 0.28385335, 0.2744509, 0.32841536...",3,3,3,4


In [6]:
data["sentences"][0]

'[\'Architecturally, the school has a Catholic character.\', "Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary.", \'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".\', \'Next to the Main Building is the Basilica of the Sacred Heart.\', \'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.\', \'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.\', \'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.\']'

In [7]:
ast.literal_eval(data["sentences"][0])

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [15]:
#get first 10 sentences
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [16]:
data

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,euclidean_dis2,minkowski_dis,chebyshev_dis,pred_idx_cos,pred_idx_euc,pred_idx_min,pred_idx_che
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.1095634 0.1142294 0.04428943 ... 0.0...,"[0.43262481689453125, 0.36864835023880005, 0.3...","[14.108408, 15.0115595, 18.043663, 13.160972, ...","[3.7561161518096924, 3.8744754791259766, 4.247...","[175.40135, 182.2885, 207.47821, 169.22188, 16...","[0.32886288, 0.3149855, 0.28589985, 0.26730207...",5,5,5,4
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.10951651 0.11030624 0.05210008 ... -0.0...,"[0.454074501991272, 0.32262009382247925, 0.355...","[12.889506, 12.285219, 16.843704, 8.361172, 11...","[3.590195894241333, 3.5050275325775146, 4.1041...","[167.5769, 162.31854, 193.53157, 130.61359, 16...","[0.31238672, 0.27531803, 0.2836463, 0.3119438,...",3,3,3,4
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.00262268 0.14930709 0.03063983 ... 0.0...,"[0.39596283435821533, 0.315723180770874, 0.346...","[11.590837, 12.180486, 16.56459, 7.4446793, 9....","[3.404531717300415, 3.4900553226470947, 4.0699...","[159.62509, 163.19696, 196.51852, 121.326935, ...","[0.26670542, 0.28385335, 0.2744509, 0.32841536...",3,3,3,4
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"['Architecturally, the school has a Catholic c...",4,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.0711433 0.05411833 -0.01395983 ... -0.0...,"[0.4900696873664856, 0.4060605764389038, 0.456...","[13.317537, 15.017247, 20.81268, 10.511387, 10...","[3.649320125579834, 3.875209093093872, 4.56209...","[170.12534, 184.275, 225.77197, 149.94484, 152...","[0.2894366, 0.26743698, 0.28701258, 0.310778, ...",3,3,3,1
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,"['Architecturally, the school has a Catholic c...",1,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[0.16131133 0.15654242 0.08214859 ... 0.06351...,"[0.4777635931968689, 0.31209897994995117, 0.36...","[14.159465, 12.130126, 17.447664, 8.731176, 12...","[3.762906551361084, 3.48283314704895, 4.177040...","[178.42073, 160.24245, 199.47891, 131.98923, 1...","[0.30424595, 0.28583807, 0.28460413, 0.3058796...",3,3,3,4
5,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,119,Rome,['The university is the major seat of the Cong...,0,"[array([ 0.07278652, 0.11468702, 0.03656113,...",[[ 0.06501997 0.13180108 -0.00197532 ... 0.0...,"[0.25486505031585693, 0.3074626326560974, 0.34...","[9.643317, 12.5554905, 16.011751, 12.947239, 1...","[3.1053690910339355, 3.543372869491577, 4.0014...","[135.46487, 168.13217, 187.52353, 169.71912, 1...","[0.4976667, 0.26768625, 0.32973778, 0.33546212...",0,0,0,4
6,The university is the major seat of the Congre...,What is the primary seminary of the Congregati...,145,Moreau Seminary,['The university is the major seat of the Cong...,5,"[array([ 0.07278652, 0.11468702, 0.03656113,...",[[ 0.0711433 0.08920594 -0.00776719 ... 0.0...,"[0.2408435344696045, 0.2794446349143982, 0.309...","[9.240584, 11.625, 14.73934, 12.287535, 9.6016...","[3.0398330688476562, 3.409545421600342, 3.8391...","[133.08464, 160.0502, 180.55904, 163.4751, 144...","[0.4976667, 0.2772242, 0.2920570

In [17]:
def create_features(data):
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    print("Finished")
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [18]:
train = create_features(data)

Finished


In [19]:
del data

In [21]:
train.head(20)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,14.108408,15.011560,18.043663,13.160972,12.855492,9.627245,15.943851,NaN,NaN,NaN,...,0.368648,0.367699,0.379501,0.374212,0.193091,0.365788,NaN,NaN,NaN,5
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601220,14.929259,NaN,NaN,NaN,...,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,NaN,NaN,NaN,2
2,11.590837,12.180486,16.564590,7.444679,9.165415,14.745420,13.291000,NaN,NaN,NaN,...,0.315723,0.346192,0.235901,0.294102,0.313559,0.317138,NaN,NaN,NaN,3
3,13.317537,15.017247,20.812680,10.511387,10.947037,16.777027,17.992474,NaN,NaN,NaN,...,0.406061,0.456177,0.353780,0.373973,0.368208,0.450643,NaN,NaN,NaN,4
4,14.159465,12.130126,17.447664,8.731176,12.842587,17.582539,15.477146,NaN,NaN,NaN,...,0.312099,0.365222,0.272937,0.406829,0.378837,0.370091,NaN,NaN,NaN,1
5,9.643317,12.555490,16.011751,12.947239,11.467084,15.898802,NaN,NaN,NaN,NaN,...,0.307463,0.343259,0.324606,0.361211,0.397475,NaN,NaN,NaN,NaN,0
6,9.240584,11.625000,14.739340,12.287535,9.601681,13.252759,NaN,NaN,NaN,NaN,...,0.279445,0.309702,0.302545,0.292370,0.320156,NaN,NaN,NaN,NaN,5
7,13.567711,13.846157,15.199486,15.804576,12.319435,14.740927,NaN,NaN,NaN,NaN,...,0.332142,0.318186,0.387593,0.362175,0.352322,NaN,NaN,NaN,NaN,2
8,15.073729,14.854823,16.454676,13.113268,13.641323,13.295512,NaN,NaN,NaN,NaN,...,0.353029,0.344759,0.314586,0.391965,0.312156,NaN,NaN,NaN,NaN,3
9,19.253742,20.027372,24.360714,19.273872,10.326595,15.372666,NaN,NaN,NaN,NaN,...,0.541039,0.582341,0.527196,0.340677,0.392760,NaN,NaN,NaN,NaN,5


In [13]:
# train.fillna(10000, inplace=True)

In [14]:
train.head(3).transpose()

,0,1,2
column_euc_0,14.108408,12.889506,11.590837
column_euc_1,15.011560,12.285219,12.180486
column_euc_2,18.043663,16.843704,16.564590
column_euc_3,13.160972,8.361172,7.444679
column_euc_4,12.855492,11.918098,9.165415
column_euc_5,9.627245,17.601220,14.745420
column_euc_6,15.943851,14.929259,13.291000
column_euc_7,NaN,NaN,NaN
column_euc_8,NaN,NaN,NaN
column_euc_9,NaN,NaN,NaN


### Fitting Multinomial Logistic Regression

### Standardize

In [15]:
train.apply(max, axis = 0)

column_euc_0    46.683550
column_euc_1    33.063713
column_euc_2    44.460903
column_euc_3    33.781600
column_euc_4    30.521135
column_euc_5    29.758953
column_euc_6    37.077620
column_euc_7          NaN
column_euc_8          NaN
column_euc_9          NaN
column_cos_0     1.000000
column_cos_1     1.307248
column_cos_2     0.869930
column_cos_3     0.930215
column_cos_4     0.985284
column_cos_5     1.000000
column_cos_6     1.000000
column_cos_7          NaN
column_cos_8          NaN
column_cos_9          NaN
target           9.000000
dtype: float64

In [44]:
subset1 = train.iloc[:,:10].fillna(60)
subset2 = train.iloc[:,10:].fillna(1)

In [45]:
 subset1.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9
0,14.108408,15.011560,18.043663,13.160972,12.855492,9.627245,15.943851,60.0,60.0,60.0
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601220,14.929259,60.0,60.0,60.0
2,11.590837,12.180486,16.564590,7.444679,9.165415,14.745420,13.291000,60.0,60.0,60.0


In [46]:
 subset2.head(3)

,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,0.432625,0.368648,0.367699,0.379501,0.374212,0.193091,0.365788,1.0,1.0,1.0,5
1,0.454075,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,1.0,1.0,1.0,2
2,0.395963,0.315723,0.346192,0.235901,0.294102,0.313559,0.317138,1.0,1.0,1.0,3


In [47]:
train2 = pd.concat([subset1, subset2],axis=1, join_axes=[subset1.index])

In [63]:
train2.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,14.108408,15.011560,18.043663,13.160972,12.855492,9.627245,15.943851,60.0,60.0,60.0,...,0.368648,0.367699,0.379501,0.374212,0.193091,0.365788,1.0,1.0,1.0,5
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601220,14.929259,60.0,60.0,60.0,...,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,1.0,1.0,1.0,2
2,11.590837,12.180486,16.564590,7.444679,9.165415,14.745420,13.291000,60.0,60.0,60.0,...,0.315723,0.346192,0.235901,0.294102,0.313559,0.317138,1.0,1.0,1.0,3


In [21]:
train2.apply(max, axis = 0)

column_euc_0    46.683550
column_euc_1    60.000000
column_euc_2    60.000000
column_euc_3    60.000000
column_euc_4    60.000000
column_euc_5    60.000000
column_euc_6    60.000000
column_euc_7    60.000000
column_euc_8    60.000000
column_euc_9    60.000000
column_cos_0     1.000000
column_cos_1     1.307248
column_cos_2     1.000000
column_cos_3     1.000000
column_cos_4     1.000000
column_cos_5     1.000000
column_cos_6     1.000000
column_cos_7     1.000000
column_cos_8     1.000000
column_cos_9     1.000000
target           9.000000
dtype: float64

In [49]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train2.iloc[:,:-1])

In [50]:
X

array([[0.2242616 , 0.19833957, 0.24788819, ..., 1.        , 1.        ,
        1.        ],
       [0.19523489, 0.14975821, 0.22637765, ..., 1.        , 1.        ,
        1.        ],
       [0.16430863, 0.14789195, 0.22137424, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.10293833, 0.09022457, 0.09922874, ..., 1.        , 1.        ,
        1.        ],
       [0.09690087, 0.13642558, 0.08488628, ..., 1.        , 1.        ,
        1.        ],
       [0.11735124, 0.19263332, 0.07391253, ..., 1.        , 1.        ,
        1.        ]])

In [51]:
train_x, test_x, train_y, test_y = train_test_split(X,
train.iloc[:,-1], train_size=0.8, random_state = 5)

In [52]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))


Multinomial Logistic regression Train Accuracy :  0.5747950819672131
Multinomial Logistic regression Test Accuracy :  0.5409836065573771


### Logistic-Regression with Root Match feature

In [34]:
predicted = pd.read_csv("train_detect_sent2.csv").reset_index(drop=True)

In [35]:
predicted = predicted[predicted["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [64]:
predicted

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,...,column_root_5,column_root_1,column_root_2,column_root_3,column_root_4,column_root_6,column_root_0,column_root_7,column_root_8,column_root_9
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.1095634 0.1142294 0.04428943 ... 0.0...,"[0.43262481689453125, 0.36864835023880005, 0.3...","[14.108408, 15.0115595, 18.043663, 13.160972, ...",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.10951651 0.11030624 0.05210008 ... -0.0...,"[0.454074501991272, 0.32262009382247925, 0.355...","[12.889506, 12.285219, 16.843704, 8.361172, 11...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.00262268 0.14930709 0.03063983 ... 0.0...,"[0.39596283435821533, 0.315723180770874, 0.346...","[11.590837, 12.180486, 16.56459, 7.4446793, 9....",...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"['Architecturally, the school has a Catholic c...",4,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.0711433 0.05411833 -0.01395983 ... -0.0...,"[0.4900696873664856, 0.4060605764389038, 0.456...","[13.317537, 15.017247, 20.81268, 10.511387, 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,"['Architecturally, the school has a Catholic c...",1,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[0.16131133 0.15654242 0.08214859 ... 0.06351...,"[0.4777635931968689, 0.31209897994995117, 0.36...","[14.159465, 12.130126, 17.447664, 8.731176, 12...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,119,Rome,['The university is the major seat of the Cong...,0,"[array([ 0.07278652, 0.11468702, 0.03656113,...",[[ 0.06501997 0.13180108 -0.00197532 ... 0.0...,"[0.25486505031585693, 0.3074626326560974, 0.34...","[9.643317, 12.5554905, 16.011751, 12.947239, 1...",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
6,The university is the major seat of the Congre...,What is the primary seminary of the Congregati...,145,Moreau Seminary,['The university is the major seat of the Cong...,5,"[array([ 0.07278652, 0.11468702, 0.03656113,...",[[ 0.0711433 0.08920594 -0.00776719 ... 0.0...,"[0.2408435344696045, 0.2794446349143982, 0.309...","[9.240584, 11.625, 14.73934, 12.287535, 9.6016...",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
7,The university is the major seat of the Congre...,What is the oldest structure at Notre Dame?,234,Old College,['The university is the major seat of the Cong...,2,"[array([ 0.07278652, 0.11468702, 0.03656113,...",[[ 0.0711433 0.05411833 0.01790375 ... 0.0...,"[0.3527829051017761, 0.3321415185928345, 0.318...","[13.567711, 13.846157, 15.199486, 15.804576, 1...",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,The university is the major seat of the Congre...,What individuals live at Fatima House at Notre...,356,Retired priests and brothers,['The university is the major seat of the Cong...,3,"[array([ 0.07278652, 0.11468702, 0.03656113,...",[[ 0.14920035 0.05743567 0.08464194 ... 0.0...,"[0.3863566517829895, 0.35302913188934326, 0.34...","[15.073729, 14.8548

In [37]:
def get_columns_from_root(train):
    
    for i in range(train.shape[0]):
        if len(ast.literal_eval(train["root_match_idx"][i])) == 0: pass
        
        else:
            for item in ast.literal_eval(train["root_match_idx"][i]):
                train.loc[i, "column_root_"+"%s"%item] = 1 + 0
    return train

In [38]:
predicted = get_columns_from_root(predicted)

In [39]:
predicted.head(3).transpose()

,0,1,2
context,"Architecturally, the school has a Catholic cha...","Architecturally, the school has a Catholic cha...","Architecturally, the school has a Catholic cha..."
question,To whom did the Virgin Mary allegedly appear i...,What is in front of the Notre Dame Main Building?,The Basilica of the Sacred heart at Notre Dame...
answer_start,515,188,279
text,Saint Bernadette Soubirous,a copper statue of Christ,the Main Building
sentences,"['Architecturally, the school has a Catholic c...","['Architecturally, the school has a Catholic c...","['Architecturally, the school has a Catholic c..."
target,5,2,3
sent_emb,"[array([ 0.05519996, 0.05013141, 0.04787038,...","[array([ 0.05519996, 0.05013141, 0.04787038,...","[array([ 0.05519996, 0.05013141, 0.04787038,..."
quest_emb,[[ 0.1095634 0.1142294 0.04428943 ... 0.0...,[[ 0.10951651 0.11030624 0.05210008 ... -0.0...,[[ 0.00262268 0.14930709 0.03063983 ... 0.0...
cosine_sim,"[0.43262481689453125, 0.36864835023880005, 0.3...","[0.454074501991272, 0.32262009382247925, 0.355...","[0.39596283435821533, 0.315723180770874, 0.346..."
euclidean_dis,"[14.108408, 15.0115595, 18.043663, 13.160972, ...","[12.889506, 12.285219, 16.843704, 8.361172, 11...","[11.590837, 12.180486, 16.56459, 7.4446793, 9...."


In [53]:
subset3 = predicted[["column_root_0","column_root_1","column_root_2","column_root_3","column_root_4","column_root_5",\
             "column_root_6","column_root_7","column_root_8","column_root_9"]]

In [54]:
subset3.fillna(0, inplace=True)

In [55]:
train3 = pd.concat([subset3, train2],axis=1, join_axes=[subset3.index])

In [65]:
train3.head(3).transpose()

,0,1,2
column_root_0,0.000000,0.000000,0.000000
column_root_1,0.000000,0.000000,1.000000
column_root_2,0.000000,0.000000,1.000000
column_root_3,0.000000,0.000000,1.000000
column_root_4,0.000000,0.000000,1.000000
column_root_5,1.000000,0.000000,1.000000
column_root_6,0.000000,0.000000,1.000000
column_root_7,0.000000,0.000000,0.000000
column_root_8,0.000000,0.000000,0.000000
column_root_9,0.000000,0.000000,0.000000


In [57]:
train3 = train3[["column_root_0","column_root_1","column_root_2","column_root_3","column_root_4","column_root_5",\
             "column_root_6","column_root_7","column_root_8","column_root_9", "column_cos_0","column_cos_1",\
           "column_cos_2","column_cos_3","column_cos_4","column_cos_5",\
             "column_cos_6","column_cos_7","column_cos_8","column_cos_9", "target"]]

In [58]:
train_x, test_x, train_y, test_y = train_test_split(train3.iloc[:,:-1],
train3.iloc[:,-1], train_size=0.8, random_state = 5)

In [59]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Logistic-Regression with Root Match feature Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Logistic-Regression with Root Match feature Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))


Logistic-Regression with Root Match feature Train Accuracy :  0.5983606557377049
Logistic-Regression with Root Match feature Test Accuracy :  0.5737704918032787


### Random Forest

In [60]:
rf = RandomForestClassifier(min_samples_leaf=8, n_estimators=60)
rf.fit(train_x, train_y)

print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, rf.predict(train_x)))
print("Random Forest Test Accuracy : ", metrics.accuracy_score(test_y, rf.predict(test_x)))

Random Forest Train Accuracy :  0.7110655737704918
Random Forest Test Accuracy :  0.6024590163934426


### XgBoost

In [61]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [3,5,10],
              "min_child_weight" : [1,5,10],
              "learning_rate": [0.07, 0.1,0.2],
               }

# run randomized search
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=5, n_jobs=-1)
grid_search.fit(train_x, train_y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] learning_rate=0.07, max_depth=3, min_child_weight=1 .............
[CV] learning_rate=0.07, max_depth=3, min_child_weight=1 .............
[CV] learning_rate=0.07, max_depth=3, min_child_weight=1 .............
[CV] learning_rate=0.07, max_depth=3, min_child_weight=5 .............
[CV]  learning_rate=0.07, max_depth=3, min_child_weight=5, score=0.5853658536585366, total=   0.5s
[CV] learning_rate=0.07, max_depth=3, min_child_weight=5 .............
[CV]  learning_rate=0.07, max_depth=3, min_child_weight=1, score=0.5382262996941896, total=   0.6s
[CV] learning_rate=0.07, max_depth=3, min_child_weight=5 .............
[CV]  learning_rate=0.07, max_depth=3, min_child_weight=1, score=0.6128048780487805, total=   0.7s
[CV] learning_rate=0.07, max_depth=3, min_child_weight=10 ............
[CV]  learning_rate=0.07, max_depth=3, min_child_weight=1, score=0.5763239875389408, total=   0.7s
[CV] learning_rate=0.07, max_depth=3, min_chil

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.4s


[CV]  learning_rate=0.07, max_depth=5, min_child_weight=5, score=0.6036585365853658, total=   0.8s
[CV] learning_rate=0.07, max_depth=5, min_child_weight=10 ............
[CV]  learning_rate=0.07, max_depth=5, min_child_weight=5, score=0.5412844036697247, total=   0.8s
[CV] learning_rate=0.07, max_depth=5, min_child_weight=10 ............
[CV]  learning_rate=0.07, max_depth=5, min_child_weight=5, score=0.5545171339563862, total=   0.8s
[CV] learning_rate=0.07, max_depth=10, min_child_weight=1 ............
[CV]  learning_rate=0.07, max_depth=5, min_child_weight=10, score=0.6036585365853658, total=   0.7s
[CV] learning_rate=0.07, max_depth=10, min_child_weight=1 ............
[CV]  learning_rate=0.07, max_depth=5, min_child_weight=10, score=0.5259938837920489, total=   0.7s
[CV] learning_rate=0.07, max_depth=10, min_child_weight=1 ............
[CV]  learning_rate=0.07, max_depth=5, min_child_weight=10, score=0.573208722741433, total=   0.8s
[CV] learning_rate=0.07, max_depth=10, min_child_

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   15.4s


[CV]  learning_rate=0.2, max_depth=5, min_child_weight=5, score=0.5792682926829268, total=   0.8s
[CV] learning_rate=0.2, max_depth=5, min_child_weight=10 .............
[CV]  learning_rate=0.2, max_depth=5, min_child_weight=1, score=0.5794392523364486, total=   0.9s
[CV] learning_rate=0.2, max_depth=5, min_child_weight=10 .............
[CV]  learning_rate=0.2, max_depth=5, min_child_weight=5, score=0.5137614678899083, total=   0.8s
[CV] learning_rate=0.2, max_depth=5, min_child_weight=10 .............
[CV]  learning_rate=0.2, max_depth=5, min_child_weight=5, score=0.573208722741433, total=   0.8s
[CV] learning_rate=0.2, max_depth=10, min_child_weight=1 .............
[CV]  learning_rate=0.2, max_depth=5, min_child_weight=10, score=0.5823170731707317, total=   0.8s
[CV] learning_rate=0.2, max_depth=10, min_child_weight=1 .............
[CV]  learning_rate=0.2, max_depth=5, min_child_weight=10, score=0.5198776758409785, total=   0.7s
[CV] learning_rate=0.2, max_depth=10, min_child_weight=1

[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   18.9s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 10], 'min_child_weight': [1, 5, 10], 'learning_rate': [0.07, 0.1, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [41]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.07, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [62]:
xg = xgb.XGBClassifier(max_depth=5)
xg.fit(train_x, train_y)

print("XgBoost Train Accuracy : ", metrics.accuracy_score(train_y, xg.predict(train_x)))
print("XgBoost Test Accuracy : ", metrics.accuracy_score(test_y, xg.predict(test_x)))

XgBoost Train Accuracy :  0.9743852459016393
XgBoost Test Accuracy :  0.6352459016393442


In [43]:
xgb.XGBClassifier??

Init signature: xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)
Docstring:     
Implementation of the scikit-learn API for XGBoost classification.

    Parameters
----------
max_depth : int
    Maximum tree depth for base learners.
learning_rate : float
    Boosting learning rate (xgb's "eta")
n_estimators : int
    Number of boosted trees to fit.
silent : boolean
    Whether to print messages while running boosting.
objective : string or callable
    Specify the learning task and the corresponding learning objective or
    a custom objective function to be used (see note below).
booster: string
    Specify which booster to use: gbtree, gblinear or dart.
nthread : int
    